```
- Copyright 2023 DeepMind Technologies Limited
- All software is licensed under the Apache License, Version 2.0 (Apache 2.0); you may not use this file except in compliance with the Apache 2.0 license. You may obtain a copy of the Apache 2.0 license at: https://www.apache.org/licenses/LICENSE-2.0
- All other materials are licensed under the Creative Commons Attribution 4.0 International License (CC-BY).  You may obtain a copy of the CC-BY license at: https://creativecommons.org/licenses/by/4.0/legalcode
- Unless required by applicable law or agreed to in writing, all software and materials distributed here under the Apache 2.0 or CC-BY licenses are distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the licenses for the specific language governing permissions and limitations under those licenses.
- This is not an official Google product
```

# Symmetric admissible set

This notebook contains

1. the *skeleton* we used for FunSearch to discover large admissible sets,
2. the *functions* discovered by FunSearch that construct large admissible sets.

## Skeleton

The commented-out decorators are just a way to indicate the main entry point of the program (`@funsearch.run`) and the function that *FunSearch* should evolve (`@funsearch.evolve`).

In [ ]:
"""Finds large admissible sets."""
import itertools
import math
import numpy as np

TRIPLES = [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 2), (0, 2, 1), (1, 1, 1), (2, 2, 2)]
INT_TO_WEIGHT = [0, 1, 1, 2, 2, 3, 3]


def expand_admissible_set(
    pre_admissible_set: list[tuple[int, ...]]) -> list[tuple[int, ...]]:
  """Expands a pre-admissible set into an admissible set."""
  num_groups = len(pre_admissible_set[0])
  admissible_set = []
  for row in pre_admissible_set:
    rotations = [[] for _ in range(num_groups)]
    for i in range(num_groups):
      x, y, z = TRIPLES[row[i]]
      rotations[i].append((x, y, z))
      if not x == y == z:
        rotations[i].append((z, x, y))
        rotations[i].append((y, z, x))
    product = list(itertools.product(*rotations))
    concatenated = [sum(xs, ()) for xs in product]
    admissible_set.extend(concatenated)
  return admissible_set


def get_surviving_children(extant_elements, new_element, valid_children):
  """Returns the indices of `valid_children` that remain valid after adding `new_element` to `extant_elements`."""
  bad_triples = set([
      (0, 0, 0), (0, 1, 1), (0, 2, 2), (0, 3, 3), (0, 4, 4), (0, 5, 5),
      (0, 6, 6), (1, 1, 1), (1, 1, 2), (1, 2, 2), (1, 2, 3), (1, 2, 4),
      (1, 3, 3), (1, 4, 4), (1, 5, 5), (1, 6, 6), (2, 2, 2), (2, 3, 3),
      (2, 4, 4), (2, 5, 5), (2, 6, 6), (3, 3, 3), (3, 3, 4), (3, 4, 4),
      (3, 4, 5), (3, 4, 6), (3, 5, 5), (3, 6, 6), (4, 4, 4), (4, 5, 5),
      (4, 6, 6), (5, 5, 5), (5, 5, 6), (5, 6, 6), (6, 6, 6)])

  # Compute.
  valid_indices = []
  for index, child in enumerate(valid_children):
    # Invalidate based on 2 elements from `new_element` and 1 element from a
    # potential child.
    if all(INT_TO_WEIGHT[x] <= INT_TO_WEIGHT[y]
           for x, y in zip(new_element, child)):
      continue
    # Invalidate based on 1 element from `new_element` and 2 elements from a
    # potential child.
    if all(INT_TO_WEIGHT[x] >= INT_TO_WEIGHT[y]
           for x, y in zip(new_element, child)):
      continue
    # Invalidate based on 1 element from `extant_elements`, 1 element from
    # `new_element`, and 1 element from a potential child.
    is_invalid = False
    for extant_element in extant_elements:
      if all(tuple(sorted((x, y, z))) in bad_triples
             for x, y, z in zip(extant_element, new_element, child)):
        is_invalid = True
        break
    if is_invalid:
      continue

    valid_indices.append(index)
  return valid_indices


def solve(n: int, w: int) -> tuple[np.ndarray, np.ndarray]:
  """Generates a symmetric constant-weight admissible set I(n, w)."""
  num_groups = n // 3
  assert 3 * num_groups == n

  # Compute the scores of all valid (weight w) children.
  valid_children = []
  for child in itertools.product(range(7), repeat=num_groups):
    weight = sum(INT_TO_WEIGHT[x] for x in child)
    if weight == w:
      valid_children.append(np.array(child, dtype=np.int32))
  valid_scores = np.array([
      priority(sum([TRIPLES[x] for x in xs], ()), n, w)
      for xs in valid_children])

  # Greedy search guided by the scores.
  pre_admissible_set = np.empty((0, num_groups), dtype=np.int32)
  while valid_children:
    max_index = np.argmax(valid_scores)
    max_child = valid_children[max_index]
    surviving_indices = get_surviving_children(pre_admissible_set, max_child,
                                               valid_children)
    valid_children = [valid_children[i] for i in surviving_indices]
    valid_scores = valid_scores[surviving_indices]

    pre_admissible_set = np.concatenate([pre_admissible_set, max_child[None]],
                                        axis=0)

  return pre_admissible_set, np.array(expand_admissible_set(pre_admissible_set))


# @funsearch.run
def evaluate(n: int, w: int) -> int:
  """Returns the size of the expanded admissible set."""
  _, admissible_set = solve(n, w)
  return len(admissible_set)


# @funsearch.evolve
def priority(el: tuple[int, ...], n: int, w: int) -> float:
  """Returns the priority with which we want to add `el` to the set."""
  return 0.0

By executing the skeleton with the trivial `priority` function in place we can for example check that in $n = 15$ dimensions and for weight $w = 10$ it only constructs an admissible set of size $1842 < 3003 = {15 \choose 10}$.

In [ ]:
for n, w in [(15, 10)]:
  print(n, w, evaluate(n, w))

15 10 1842


## Discovered function that builds a full-size $I(15, 10)$ admissible set

This function discovered by FunSearch results in a full-sized admissible set $I(15, 10)$, i.e. of size ${15 \choose 10} = 3003$:

In [ ]:
def priority(el: tuple[int, ...], n: int, w: int) -> float:
  score = 0.0
  for i in range(n):
    if el[i] < el[i - 1]:
      score += 1
    elif el[i] < el[i - 2]:
      score += 0.05
    elif el[i] < el[i - 3]:
      score -= 0.05
    elif el[i] < el[i - 4]:
      score += 0.01
    elif el[i] < el[i - 5]:
      score -= 0.01
    elif el[i] < el[i - 6]:
      score += 0.001
    else:
      score += 0.005

  for i in range(n):
    if el[i] == el[i - 1]:
      score -= w
    elif el[i] == 0 and i != n - 1 and el[i + 1] != 0:
      score += w
    if el[i] != el[i - 1]:
      score += w

  for i in range(n):
    if el[i] < el[i - 1]:
      if el[i] == 0:
        score -= w
  return score


pre_admissible_15_10, admissible_15_10 = solve(15, 10)
assert admissible_15_10.shape == (math.comb(15, 10), 15)
assert pre_admissible_15_10.shape == (101, 5)

## Discovered function that builds a size $43\,596$ admissible set in $\mathcal{A}(21, 15)$

This admissible set implies an improved lower bound of $2.220041$ on the cap set capacity.

*Note*: After uncommenting the invocation below, executing this cell can take ~15 minutes.

In [ ]:
def priority(el: tuple[int, ...], n: int, w: int) -> float:
  score = 0
  coeff = 0
  for pos, x in zip(range(n), el):
    y = (el[(pos + 1) % n] - el[(pos)]) % n
    z = (el[(pos + 2) % n] - el[(pos)]) % n
    p = (el[(pos - 1) % n] + 1) % n

    u = (el[(pos - 2) % n] + 1) % n
    v = (el[(pos + 3) % n] + 1) % n

    score += 3 * p * (p + coeff) * (p + w) + (p + coeff)**2 * (w + 1)
    score += 2 * p * v * (p + w) + v * z * (-1 + w) - (p + coeff) * (-1 + w)
    score += v * (u + w) + u + 3 * u * y * (1 + w) + u * z * (w - 1) - (p + coeff) * (w - 1)
    score += (1 + w)**6 * 3 * coeff**2

  return score


# Uncomment to execute; note it can take ~15 minutes to run this.
# pre_admissible_21_15, admissible_21_15 = solve(21, 15)
# assert admissible_21_15.shape == (43_596, 21)
# assert pre_admissible_21_15.shape == (308, 7)

## Discovered function that builds a size $237\,984$ admissible set in $\mathcal{A}(24, 17)$

This admissible set implies a new state-of-the-art lower bound of $2.220234$ on the cap set capacity.

*Note*: Running `solve(24, 17)` takes ~7 hours. In practice we used a C++ reimplementation of the function `get_surviving_children` to speed things up. Note that we provide the result of running this code as standalone files `pre_admissible_set_for_n24_w17_size237984.txt` and `admissible_set_n24_w17_size237984.txt` in the same directory as this notebook.

In [ ]:
def priority(el: tuple[int, ...], n: int, w: int) -> float:
  result = 0.0
  for i in range(n):
    n_violations = 0

    if el[i] < el[i - 1]:
      result += (el[i - 1] ** 0.5) * w ** 2 / (6 * 6)
      n_violations += 1

    if el[i] < el[i - 2]:
      result += el[i - 2] ** 0.5
      n_violations += 1

    if el[i - 1] != 0:
      result -= (el[i] - el[i - 1]) * w ** 2 / (6 * 3)
      n_violations += 2

    if el[i - 2] != 0:
      result -= (el[i] - el[i - 2]) * w ** 2 / (6 * 6) * (0.95 ** n_violations)
      n_violations += 1

    result -= (0.02 ** el[i]) * (el[i] - el[i - 8])

  return result


# Executing this would take ~7 hours.
# pre_admissible_24_17, admissible_24_17 = solve(24, 17)
# assert admissible_24_17.shape == (237_984, 24)
# assert pre_admissible_24_17.shape == (736, 8)